# Crowling Berita

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import uuid # Meskipun tidak digunakan untuk ID dari web, bisa untuk ID unik jika diperlukan

In [2]:
# URL homepage CNN Indonesia
url = "https://www.cnnindonesia.com/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0"}

# Ambil halaman utama
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

data = []

# Loop semua link artikel di homepage
for artikel in soup.select("article a"):
    link = artikel.get("href")
    judul = artikel.get_text(strip=True)

    # Skip jika link kosong atau bukan http
    if not link or not link.startswith("http"):
        continue

    try:
        # Ambil halaman detail berita
        resp_detail = requests.get(link, headers=headers)
        soup_detail = BeautifulSoup(resp_detail.text, "html.parser")

        # --- PERUBAHAN DIMULAI DI SINI ---
        
        # Ambil ID Berita dari link
        id_berita = None
        try:
            # Contoh path: /ekonomi/20250910153012-92-1234567/nama-artikel
            # Kita ambil bagian sebelum nama artikel, yaitu '20250910153012-92-1234567'
            id_segment = link.split("/")[-2]
            # Kemudian kita pisah dengan '-' dan ambil bagian terakhirnya
            id_berita = id_segment.split("-")[-1]
        except (IndexError, AttributeError):
            # Jika struktur URL berbeda dan gagal, ID akan tetap None
            id_berita = None

        # Ambil kategori dari link (path pertama setelah domain)
        kategori = None
        try:
            path = link.replace("https://www.cnnindonesia.com/", "")
            kategori = path.split("/")[0] if path else None
        except Exception:
            kategori = None

        # Ambil isi berita
        paragraf = [p.get_text(strip=True) for p in soup_detail.select("div.detail-text p")]
        isi = " ".join(paragraf)

        if isi:  # hanya simpan kalau ada isi berita
            data.append({
                "id_berita": id_berita, # <-- Kolom baru ditambahkan di sini
                "judul": judul,
                "kategori": kategori,
                "isi": isi,
                "link": link
            })
            
        # --- PERUBAHAN SELESAI DI SINI ---

    except Exception as e:
        print(f"Gagal ambil {link}: {e}")

# Simpan ke dataframe Pandas
df = pd.DataFrame(data)

# Simpan ke file CSV
df.to_csv("berita_cnn_dengan_id.csv", index=False, encoding="utf-8-sig")

# Tampilkan hasil di notebook
pd.set_option("display.max_colwidth", 100)  # biar isi tidak kepotong
print("Proses selesai. Berikut adalah contoh datanya:")
display(df)

Proses selesai. Berikut adalah contoh datanya:


,id_berita,judul,kategori,isi,link
0,1272341,Indra Sjafri Yakin Vanenburg Bisa Bawa Timnas U-23 Meroket,olahraga,Mantan direktur teknik PSSIIndra Sjafrimemberikan dukungan kepada pelatihGerald Vanenburgsetelah...,https://www.cnnindonesia.com/olahraga/20250910222925-142-1272341/indra-sjafri-yakin-vanenburg-bi...
1,1272305,Pelatih Korea: Vanenburg Butuh Waktu di Timnas Indonesia U-23,olahraga,Pelatih Korea Selatan Lee Min Sung menilaiGerald Vanenburgsebagai sosok yang kuat dan bagus namu...,https://www.cnnindonesia.com/olahraga/20250910200409-142-1272305/pelatih-korea-vanenburg-butuh-w...
2,1272266,01Sara Keponakan Prabowo Mundur dari DPRNasional,nasional,Politikus Partai GerindraRahayu Saraswati Djojohadikusumoalias Sara memutuskan mundur sebagai an...,https://www.cnnindonesia.com/nasional/20250910182921-32-1272266/sara-keponakan-prabowo-mundur-da...
3,1272319,02Prabowo Setujui Ide Menkeu Tarik Rp200 T Uang Pemerintah di BIEkonomi,ekonomi,Presiden Prabowo Subianto menyetujui ide Menteri KeuanganPurbaya Yudhi Sadewauntuk menarik Rp200...,https://www.cnnindonesia.com/ekonomi/20250910204701-532-1272319/prabowo-setujui-ide-menkeu-tarik...
4,1272280,03VIDEO: Menkes Baru Swedia Ambruk saat Jumpa Pers PerdanaInternasional,internasional,"Menteri Kesehatan Swedia yang baru dipilih, Elisabeth Lann, pingsan saat menggelar jumpa pers pe...",https://www.cnnindonesia.com/internasional/20250910185359-139-1272280/video-menkes-baru-swedia-a...
5,1272135,"04Berstatus Terpidana, Silfester Matutina Masih Jadi Komisaris ID FoodEkonomi",ekonomi,"Silfester Matutina, terpidana kasus dugaan pencemaran nama baik terhadap Jusuf Kalla, ternyata m...",https://www.cnnindonesia.com/ekonomi/20250910135552-92-1272135/berstatus-terpidana-silfester-mat...
6,1272234,05Timnas Futsal Indonesia Siap Tumpas Denmark di Final CFAOlahraga,olahraga,"PelatihTimnas Futsal Indonesiadi CFA International Tournament 2025, Amril Daulay, mengaku siap m...",https://www.cnnindonesia.com/olahraga/20250910165232-142-1272234/timnas-futsal-indonesia-siap-tu...
7,1272341,06Indra Sjafri Yakin Vanenburg Bisa Bawa Timnas U-23 MeroketOlahraga,olahraga,Mantan direktur teknik PSSIIndra Sjafrimemberikan dukungan kepada pelatihGerald Vanenburgsetelah...,https://www.cnnindonesia.com/olahraga/20250910222925-142-1272341/indra-sjafri-yakin-vanenburg-bi...
8,1271984,"Bali Diterjang Banjir Besar, Dua Warga Meninggal Dunia",nasional,"Banjirbesar melandaBali, menerjang sejumlah daerah seperti Kabupaten Jembrana dan Kota Denpasar ...",https://www.cnnindonesia.com/nasional/20250910093338-20-1271984/bali-diterjang-banjir-besar-dua-...
9,1271909,"Timnas Indonesia era Kluivert: Inverted Fullback, Ball Winner, Mezzala",olahraga,Tak berlebihan kiranya jikaPatrick Kluivertdisebut mulai memperlihatkan jati dirinya diTimnas In...,https://www.cnnindonesia.com/olahraga/20250910061945-142-1271909/timnas-indonesia-era-kluivert-i...
